# A (climate) data manager example notebook
- connect to (management) databases
- do things on file system things
- interact with databases
- look into and check climate data
- change metadata and associate data with identifiers
- ...

In [1]:
%load_ext sql

/home/stephan/anaconda/envs/scientific1/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/stephan/anaconda/envs/scientific1/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
%sql sqlite:///ingest.db

u'Connected: None@ingest.db'

In [3]:
%%sql
CREATE TABLE ingest_files (aggregation_name, filename, date, checksum);
CREATE TABLE ingest_aggregations(origin,contact,location);
INSERT INTO ingest_files VALUES ('CMIP5/MPI-M/day/psl/', 'pr_day_MPI-ESM-LR_1pctCO2_r1i1p1_18500101-18591231.nc', '2012-11-02','md5:89A5543');
INSERT INTO ingest_aggregations VALUES ('CMIP5/MPI-M/day/psl/','ho@mpi-m.de','/gpfs750/transfer/CMIP5');

Done.
Done.
1 rows affected.
1 rows affected.


[]

In [6]:
result = %sql select * from ingest_files where filename='pr_day_MPI-ESM-LR_1pctCO2_r1i1p1_18500101-18591231.nc'

(aggregation_name,filename,ingest_date,checksum) = result[0]
print aggregation_name

Done.
CMIP5/MPI-M/day/psl/


In [ ]:
%%bash
pwd
ls -al ~stephan/Repos/Data


In [ ]:
# docu: https://code.zmaw.de/projects/cdo/embedded/index.html
from cdo import *
cdo = Cdo()

In [ ]:
input1="/home/stephan/Data/pr_day_MPI-ESM-LR_1pctCO2_r1i1p1_18500101-18591231.nc"
input2="/home/stephan/Data/pr_day_MPI-ESM-LR_1pctCO2_r1i1p1_18600101-18691231.nc"
cdo.showname(input=input1)

In [ ]:
cdo.griddes(input=input1)

In [ ]:
cdo.pardes(input=input1)
#cdo.griddes(input=input1)

In [ ]:
cdo.showstdname(input=input1)

In [ ]:
cdo.showname(input=input1)

In [ ]:
cdo.sinfo(input=input1)

In [ ]:
cdo.infon(input=input1)

In [ ]:
cdo.diff(input=(input1,input2))

In [ ]:
cdo.eca_cdd(input=input1,output="/home/stephan/Data/drydays.nc")